In [ ]:
import os
import sys
import time
import json
import pandas as pd
import numpy as np
import datetime as dtm
import matplotlib.pyplot as plt

# AFC
from afc.ctrlWrapper import Controller, make_inputs
from afc.defaultConfig import default_parameter
from afc.utility.plotting import plot_standard1

# pvlib
import warnings
warnings.filterwarnings('ignore', message="The forecast module algorithms and features are highly experimental.")
warnings.filterwarnings('ignore', message="The HRRR class was deprecated in pvlib 0.9.1 and will be removed in a future release.")
from pvlib.forecast import HRRR

In [ ]:
def get_weather_forecast(parameter):
#     wf = pd.DataFrame(index=pd.date_range('09-01-2022', '09-02-2022', freq='5T'),
#                       columns=['DryBulb','DNI','DHI','Wspd']+['GHI']).fillna(1)

    tz = 'US/Pacific'
    now = dtm.datetime.now().replace(minute=0, second=0, microsecond=0)
    start_time = pd.Timestamp(now, tz='UTC').tz_convert(tz)
    final_time = start_time + pd.Timedelta(hours=24)

    forecaster = HRRR()
    query_variables = list(forecaster.variables.values())
    query_variables.remove('Temperature_height_above_ground')
    query_variables.remove('v-component_of_wind_height_above_ground')
    query_variables.remove('u-component_of_wind_height_above_ground')

    wf1 = forecaster.get_data(parameter['radiance']['location']['latitude'],
                              -1*parameter['radiance']['location']['longitude'],
                              start_time, final_time,
                              query_variables=query_variables)
    
    forecaster = HRRR()
    wf2 = forecaster.get_data(parameter['radiance']['location']['latitude'],
                              -1*parameter['radiance']['location']['longitude'],
                              start_time, final_time,
                              query_variables=['Temperature_height_above_ground'])
    
    wf = pd.concat([wf1, wf2], axis=1)
    #wf.index = wf.index.tz_localize(None)
    
    # process
    wf['wind_speed_u'] = 0
    wf['wind_speed_v'] = 0
    # duplicate last beacuse of bug in pvlib
    wf.loc[wf.index[-1]+pd.DateOffset(hours=1), :] = wf.iloc[-1]
    data = forecaster.process_data(wf)
    data = data.loc[wf.index[:-1]]
    data.index = data.index.tz_localize(None)
    
    # rename
    col_map = {'temp_air': 'DryBulb', 'wind_speed': 'Wspd', 'ghi': 'GHI', 'dni': 'DNI', 'dhi': 'DHI'}
    data = data.rename(columns=col_map)
    return data[col_map.values()]

def plot_result(wf, ctrl):
    df = pd.DataFrame(ctrl.get_output(keys=['df_output'])['df_output'])
    df.index = pd.to_datetime(df.index, unit='ms')
    plot_standard1(pd.concat([wf, df], axis=1).ffill())
    
tint_map = {3: 0, 2: 28, 1: 49, 0: 82}

### Run the Controller

In [ ]:
# Initialize Controller
ctrl = Controller()
parameter = default_parameter(precompute_radiance=False)

print('uShade: bot, mid, top')
run = True
while run:
    
    # now
    now = pd.Timestamp(dtm.datetime.now().replace(microsecond=0), tz='UTC').tz_convert('US/Pacific').tz_localize(None)
    
    # Get Weather Forecast
    wf = get_weather_forecast(parameter)
    
    # Make Inputs
    inputs = make_inputs(parameter, wf)
    
    # Query Controller and get Outputs
    ctrl.do_step(inputs=inputs)
    print(now, ctrl.get_output(keys=['uShade', 'duration']))

    # Set control
    with open('control.json', 'w') as f:
        u = ctrl.get_output(keys=['uShade'])['uShade'] # bot, mid, top
        u = {6-k:tint_map[int(v)] for k,v in enumerate(u)} # 4: 'RmB-top', 5: 'RmB-mid', 6: 'RmB-bot'
        f.write(json.dumps(u))
    
    # Store log
    now_str = str(now).replace(":","").replace(" ","T")
    wf.to_csv(f'log/wf_{now_str}.csv')
    pd.DataFrame(ctrl.get_output(keys=['df_output'])['df_output']).to_csv(f'log/ctrl_{now_str}.csv')
    with open(f'log/stats_{now_str}.json', 'w') as f:
        f.write(json.dumps(ctrl.get_output(keys=['rad_duration','varts_duration','optall_duration',
                                                 'glare_duration','opt_duration','outputs_duration',
                                                 'duration','opt_objective','opt_duration','opt_termination'])))
    
    # Plot
#     plot_result(wf, ctrl)
    
#     time.sleep((int(time.time() / (60*60)) + 1) * 60*60 - time.time()) # hourly timestep
    time.sleep((int(time.time() / (5*60)) + 1) * 5*60 - time.time()) # 5 min timestep